# Value Iteration - Implementation

Value iteration is a method of computing an optimal MDP policy and its value.

#### Import libraries

In [1]:
import numpy as np

#### Initialise parameters

In [2]:
REWARD = -0.1
GAMMA = 0.7
DISCOUNT = 0.99 

ACTIONS = [(1, 0), (0, -1), (-1, 0), (0, 1)]  # actions which can be D, L, U, R
NUM_ACTIONS = len(ACTIONS)

# Grid size based on amount of rows and columns
ROW = 4
COL = 4

# Grid with terminal states
u = [[0, 0, 0, 40],
     [0, 0, 0, 0],
     [0, 0, 0, 0],
     [10, 0, 0, 0]]

In [15]:
def maze_grid(arr, policy=False):
    """This function initializes the maze grid with all of the 
    given rewards per state and prints it out nicely.    
    The grid has 4 rows and 4 columns which is set in the cell above.
    """
    res = ""
    for r in range(ROW):
        res += "|"
        for c in range(COL):
            if r == 0 and c == 3:
                val = '\u2610'
            elif r == 3 and c == 0:
                val = '\u2610'
            elif r == 3 and c == 2:
                val += " S"
            else:
                if policy:
                    val = ['\u2193', '\u2190', '\u2191', '\u2192'][arr[r][c]] # DOWN/LEFT/UP/RIGHT
                else:
                    val = str(arr[r][c])
            res += " " + val[:5].ljust(5) + " |" 
        res += "\n"
    print(res)

#### 'Visualize' the maze grid

In [16]:
maze_grid(u)

| 0     | 0     | 0     | ☐     |
| 0     | 0     | 0     | 0     |
| 0     | 0     | 0     | 0     |
| ☐     | 0     | 0 S   | 0     |



In [5]:
def get_utility(rewards, r, c, action):
    """This function gets the utility of the state reached 
    by performing the given action from the given state
    """
    dr, dc = ACTIONS[action]
    newR = r + dr
    newC = c + dc
    if newR < 0 or newC < 0 or newR >= ROW or newC >= COL or (newR == 3 and newC == 2): 
        return rewards[r][c]
    else:
        return rewards[newR][newC]

In [6]:
def calculate_utility(rewards, r, c, action):
    """This function calculates the utility of a state given 
    an action
    """
    if r == 3 and c == 1:
        u = -2
    elif r == 1 and c == 2:
        u = -10
    elif r == 1 and c == 3:
        u = -10
    else:
        u = REWARD
                
    u += 0.1 * GAMMA * get_utility(rewards, r, c, (action-1)%4)
    u += 0.7 * GAMMA * get_utility(rewards, r, c, action)
    u += 0.1 * GAMMA * get_utility(rewards, r, c, (action+1)%4)
    
    return u

In [8]:
def value_iteration(rewards):
    """This function initializes the maze grid with all of the 
    given rewards per state and prints it out nicely.    
    The grid has 4 rows and 4 columns which is set in the cell above.
    """
    print("During the value iteration:\n")
    while True:
        next_utility = [[0, 0, 0, 40],
                        [0, 0, 0, 0],
                        [0, 0, 0, 0],
                        [10, 0, 0, 0]]
        error = 0
        for r in range(ROW):
            for c in range(COL):
                if (r == 0 and c == 3) or (r == 3 and c == 0) or (r == 3 and c == 1):
                    continue
                next_utility[r][c] = max([calculate_utility(rewards, r, c, action) for action in range(NUM_ACTIONS)]) # Bellman update
                error = max(error, abs(next_utility[r][c]-rewards[r][c]))
        rewards = next_utility
        maze_grid(rewards)
        if error < ((1-GAMMA) / GAMMA):
            break
    return rewards

In [20]:
def optimal_policy(rewards):
    """This function gets the optimal policy from U
    """
    policy = [[-1,-1,-1,40],
              [-1,-1,-10,-10],
              [-1,-1,-1,-1],
              [10,-2,-1,-1]]

    for r in range(ROW):
        for c in range(COL):
            if (r == 0 and c == 3) or (r == 3 and c == 0) or (r == 3 and c == 1):
                continue
            # Choose the action that maximizes the utility
            maxAction, maxU = None, -float("inf")
            for action in range(NUM_ACTIONS):
                u = calculate_utility(rewards, r, c, action)
                if u > maxU:
                    maxAction, maxU = action, u
                    print("MA {}".format(maxAction))
                    print("MU {}".format(maxU))
            policy[r][c] = maxAction
    return policy

#### Print optimal policy 

In [21]:
# Print the initial environment
print("The initial maze grid is:\n")
maze_grid(u)

# Value iteration
rewards = value_iteration(u)
print('Rewards: {} \n'.format(rewards))

# Get the optimal policy from U and print it
policy = optimal_policy(rewards)
print("The optimal policy is:\n")
maze_grid(policy, True)

The initial maze grid is:

| 0     | 0     | 0     | ☐     |
| 0     | 0     | 0     | 0     |
| 0     | 0     | 0     | 0     |
| ☐     | 0     | 0 S   | 0     |

During the value iteration:

| -0.1  | -0.1  | 19.49 | ☐     |
| -0.1  | -0.1  | -10.0 | 9.599 |
| 4.8   | -0.1  | -0.1  | -0.1  |
| ☐     | 2.899 | 2.899 | -0.1  |

| -0.16 | 9.440 | 20.16 | ☐     |
| 2.237 | -0.16 | 0.219 | 9.572 |
| 2.923 | 2.447 | -0.16 | 4.589 |
| ☐     | 0.890 | 0.890 | -0.16 |

| 4.671 | 10.43 | 20.92 | ☐     |
| 1.478 | 4.698 | 0.539 | 10.28 |
| 1.900 | 1.383 | 2.153 | 4.900 |
| ☐     | -0.14 | -0.14 | 2.126 |

| 5.441 | 11.21 | 21.00 | ☐     |
| 2.662 | 5.152 | 1.303 | 10.35 |
| 1.091 | 2.485 | 2.489 | 5.433 |
| ☐     | -0.95 | -0.95 | 2.598 |

| 5.961 | 11.33 | 21.06 | ☐     |
| 3.113 | 5.672 | 1.376 | 10.41 |
| 1.454 | 2.675 | 2.827 | 5.529 |
| ☐     | -0.76 | -0.76 | 2.926 |

| 6.090 | 11.41 | 21.07 | ☐     |
| 3.436 | 5.769 | 1.446 | 10.42 |
| 1.714 | 2.979 | 2.904 | 5.589 |
| ☐     | -0.70 | -0